In [2]:
# path1 = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_499_500_501/peak_area_blank_filtered/499_500_501_area.csv'
# path1 = 'D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/NEW_acquisition_20241018/Area_1_2024_10_21_14_51_42.csv'
path2 = 'D:/UCSF_postdoc_topic/ESI_/ENTACT/first_ENTACT_mixtures_mzlist.csv'
path4 = 'D:/UCSF_postdoc_topic/ESI_/EPA_pesticide_DTSC_phthalates/Pesticide_phthlates_mzlist.csv'
path5 = 'D:/UCSF_postdoc_topic/ESI_/EPA_pesticide_DTSC_phthalates/IS_for_eachbatch_mzlist.csv'
# output_fd = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_502_504_506/peak_area_with_blank_filter/'
RTpath6 = 'D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/ENTACTmix_RT_manualcheck.csv'
RTpath7 = 'D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/Othermixture_RT_manualcheck.csv'
output_fd = 'D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/'
peakareadata = 'D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/SQ_oct_ENTACT_EPA_PHTH_pos/Area_1_2024_10_31_14_05_32_ENACT_pesticide_phth_pos.csv'

In [ ]:
import pandas as pd

# Step 1: Load the Data
# Load the raw LC-MS data file
#path1 = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_499_500_501/peak_area_blank_filtered/499_500_501_area.csv'
# path1 = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_502_504_506/peak_area_with_blank_filter/502_504_506_area.csv'
path1 = 'D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/NEW_acquisition_20241018/Area_1_2024_10_21_14_51_42.csv'
feature_data = pd.read_csv(path1)

# Step 2: Extract Required Columns
# Extract 'Average Rt(min)', 'Alignment ID', 'Average Mz', and sample columns
columns_to_keep = ['RT', 'AlignID', 'Average_MZ']
# sample_columns = [col for col in feature_data.columns if col.startswith(('pos_EPA502', 'pos_EPA504', 'pos_EPA506',
#                                                                          'pos_EPA499', 'pos_EPA500', 'pos_EPA501',
#                                                                          'pos_EPA507', 'pos_EPA508'))]


#For new acquisition 'pos_EPA499_100X'
columns_to_keep.extend(sample_columns)
filtered_data = feature_data.rename(columns={'Average Rt(min)': 'RT', 'Alignment ID': 'AlignID', 'Average Mz': 'Average_MZ'})[columns_to_keep]

# Step 3: Print the Head of the DataFrame
print(filtered_data.head())
# filtered_data.to_csv('D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_502_504_506/peak_area_with_blank_filter/502_504_506_area_filtered.csv', index=False)
filtered_data.to_csv('D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/499_500_501_502_504_506_507_508_area_filtered.csv', index=False)

      RT  AlignID  Average_MZ  pos_EPA499_100X  pos_EPA500_100X  \
0  1.566        0   100.07566          5941903          4890390   
1  0.750        1   100.07571          8426027          4117234   
2  0.081        2   100.07571          3095804          5367883   
3  1.308        3   100.07574         14109951          3013204   
4  3.261        4   100.07581          3205843         11335106   

   pos_EPA501_100X  pos_EPA502_100X  pos_EPA504_100X  pos_EPA506_100X  \
0          2719208          5288036          3844434          3751659   
1          5858812         11279772         12348388         11763927   
2          3632987          4202877          3622470          6990118   
3         15122946          2850842         14167199         13391818   
4          3284039          6939958         13422485         19663452   

   pos_EPA507_100X  pos_EPA508_100X  
0          6022275          6170355  
1         13112304         13541033  
2          3198011          5714371  
3     

In [42]:
import os
import pandas as pd
import numpy as np

# Step 1: Load the Cleaned Data
# Load the cleaned LC-MS data file
# path1 = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_502_504_506/peak_area_with_blank_filter/502_504_506_area_filtered.csv'
# filtered_data = pd.read_csv(path1)

# Step 2: Separate Data by Groups
# Extract the data for each group separately
# unique_groups = filtered_data.columns.str.extract(r'(E\d{3})')[0].dropna().unique()
unique_groups = filtered_data.columns.str.extract(r'(pos_EPA\d{3})')[0].dropna().unique()
print(f"Unique groups found: {unique_groups}")

# Step 3: Iterate over each group
for group in unique_groups:
    # Extract group-specific data, keep 'Average_MZ' and 'RT' columns
    group_data = filtered_data.filter(like=group)
    group_data = pd.concat([filtered_data[['Average_MZ', 'RT']], group_data], axis=1)
    print(f"Processing group: {group}, Columns extracted: {group_data.columns.tolist()}")

    # Step 4: Calculate average intensity and CV% for each treatment and add to the data
    # for treatment in ["100x", "200x", "1000x", "5000x", "25000x"]:
    for treatment in ["100X"]:
        treatment_cols = [col for col in group_data.columns if f"_{treatment}_" in col]
        if len(treatment_cols) == 0:
            group_data[f"avg_intensity_{treatment}"] = np.nan
            group_data[f"cv_percent_{treatment}"] = np.nan
            continue
        treatment_data = group_data[treatment_cols]
        cv_percent = treatment_data.std(axis=1) / treatment_data.mean(axis=1) * 100
        avg_intensity = treatment_data.mean(axis=1)
        group_data[f"avg_intensity_{treatment}"] = np.where(cv_percent <= 30, avg_intensity, np.nan)
        group_data[f"cv_percent_{treatment}"] = cv_percent

    # Step 5: Export Group Data
    # output_folder = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_502_504_506/peak_area_with_blank_filter/'
    output_folder = 'D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/'
    os.makedirs(output_folder, exist_ok=True)
    filtered_data_path = os.path.join(output_folder, f"filtered_feature_table_{group}.csv")
    group_data.to_csv(filtered_data_path, index=False)
    print(f"Filtered data saved for group {group}: {filtered_data_path}")

Unique groups found: ['pos_EPA499' 'pos_EPA500' 'pos_EPA501' 'pos_EPA502' 'pos_EPA504'
 'pos_EPA506' 'pos_EPA507' 'pos_EPA508']
Processing group: pos_EPA499, Columns extracted: ['Average_MZ', 'RT', 'pos_EPA499_100X']
Filtered data saved for group pos_EPA499: D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/filtered_feature_table_pos_EPA499.csv
Processing group: pos_EPA500, Columns extracted: ['Average_MZ', 'RT', 'pos_EPA500_100X']
Filtered data saved for group pos_EPA500: D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/filtered_feature_table_pos_EPA500.csv
Processing group: pos_EPA501, Columns extracted: ['Average_MZ', 'RT', 'pos_EPA501_100X']
Filtered data saved for group pos_EPA501: D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/filtered_feature_table_pos_EPA501.csv
Processing group: pos_EPA502, Columns extracted: ['Average_MZ', 'RT', 'pos_EPA502_100X']
Filtered data saved for group pos_EPA502: D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/filtered_feature_table_pos_EPA502.csv
Proc

In [43]:
import pandas as pd

# Step 1: Load the Target m/z Data
# Load the target m/z data file
path = 'D:/UCSF_postdoc_topic/ESI_/ENTACT/first_ENTACT_mixtures_mzlist.csv'
target_mz_data = pd.read_csv(path)

# Step 2: Divide Concentration Columns by 1000
# Assuming the concentration columns are named 'conc1', 'conc2', etc.
concentration_columns = [col for col in target_mz_data.columns if 'conc' in col]
target_mz_data[concentration_columns] = target_mz_data[concentration_columns] / 1000

# Step 3: Save the Processed Data
# Save the preprocessed target m/z data to a new CSV file
# output_path = 'D:/UCSF_postdoc_topic/ESI_/ENTACT/processed_target_mzlist.csv'
# target_mz_data.to_csv(output_path, index=False)
# print(f"Preprocessed data saved to: {output_path}")

In [44]:
target_mz_data.head()

,group_id,DTXSID,compound_id,SMILES,MOLECULAR_FORMULA,m/z,conc1,conc2,conc3,conc4,conc5
0,E499,DTXSID5047328,MK-274,O.[K+].NC(=O)C1=NC(=N[N-]1)C1=CC=CC(=C1)C1=CC(...,C18H13F6KN4O3,486.052891,0.2,0.1,0.02,0.004,0.0008
1,E499,DTXSID5027198,N-Methylphthalimide,CN1C(=O)C2=C(C=CC=C2)C1=O,C9H7NO2,161.047678,0.2,0.1,0.02,0.004,0.0008
2,E499,DTXSID9032113,Tebuconazole,CC(C)(C)C(O)(CCC1=CC=C(Cl)C=C1)CN1C=NC=N1,C16H22ClN3O,307.145140,0.2,0.1,0.02,0.004,0.0008
3,E499,DTXSID6024466,4-Aminobenzoic acid,NC1=CC=C(C=C1)C(O)=O,C7H7NO2,137.047678,0.2,0.1,0.02,0.004,0.0008
4,E499,DTXSID4040002,Monobutyl phthalate,CCCCOC(=O)C1=C(C=CC=C1)C(O)=O,C12H14O4,222.089209,0.2,0.1,0.02,0.004,0.0008


In [40]:
import os
import pandas as pd
import numpy as np

# Step 1: Load Processed Data and Targeted m/z List
# Load the processed LC-MS data file
# change the batch ID to each results
# processed_data_path = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_502_504_506/peak_area_with_blank_filter/filtered_feature_table_E506.csv'
processed_data_path = 'D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/filtered_feature_table_pos_EPA506.csv'
processed_data = pd.read_csv(processed_data_path)

# Load the targeted m/z list
targeted_mz_list_path = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_502_504_506/peak_area_with_blank_filter/506_batch_target_mzlist.csv'
targeted_mz_list = pd.read_csv(targeted_mz_list_path)

# Step 2: Initialize an Empty DataFrame to Store Results
combined_results = pd.DataFrame()

# Step 3: Iterate Over Each Targeted m/z
for idx, row in targeted_mz_list.iterrows():
    target_mz = row['m/z']
    concentrations = [row['conc1'], row['conc2'], row['conc3'], row['conc4'], row['conc5']]

    # Step 4: Find m/z Matches Within 10 ppm
    ppm_tolerance = 50
    matching_rows = processed_data.copy()
    matching_rows['ppm'] = abs((matching_rows['Average_MZ'] - target_mz) / target_mz) * 1e6
    matching_rows = matching_rows[matching_rows['ppm'] <= ppm_tolerance]

    # Step 5: Add Metadata from Targeted m/z List to Matching Rows
    for col in ['group_id', 'DTXSID', 'compound_id', 'SMILES', 'MOLECULAR_FORMULA','m/z']:
        matching_rows[col] = row[col]

    # Step 6: Calculate LogRF for Each Treatment/Concentration Pair with CV Filtering
    logRF_columns = []
    for i, (treatment, conc) in enumerate(zip(["100x", "200x", "1000x", "5000x", "25000x"], concentrations), start=1):
        avg_intensity_col = f"avg_intensity_{treatment}"
        cv_col = f"cv_percent_{treatment}"
        logRF_col = f'LogRF_{i}'
        if avg_intensity_col in matching_rows.columns and cv_col in matching_rows.columns and conc != 0:
            matching_rows[logRF_col] = np.where(matching_rows[cv_col] <= 30, np.log10(matching_rows[avg_intensity_col] / conc), np.nan)
        else:
            matching_rows[logRF_col] = np.nan
        logRF_columns.append(logRF_col)

    # Step 7: Calculate Average LogRF if More Than 3 Values are Available
    matching_rows['average_LogRF'] = matching_rows[logRF_columns].apply(lambda x: x[x.notna()].mean() if x.count() >= 3 else np.nan, axis=1)

    # Step 8: Combine Results
    combined_results = pd.concat([combined_results, matching_rows], ignore_index=True)

# Step 9: Filter Combined Results to Keep Only Rows with Valid average_LogRF and Lowest ppm
filtered_results = combined_results[combined_results['average_LogRF'].notna()]
filtered_results = filtered_results.sort_values(by=['m/z', 'ppm']).drop_duplicates(subset='m/z', keep='first')

# Step 10: Export Combined Results
# output_folder = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_499_500_501/peak_area_blank_filtered/targeted_res/'
output_folder = 'D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/'
os.makedirs(output_folder, exist_ok=True)
group_id = filtered_results['group_id'].iloc[0] if not filtered_results.empty else 'combined'
combined_results_path = os.path.join(output_folder, f"combined_targeted_mz_results_{group_id}.csv")
filtered_results.to_csv(combined_results_path, index=False)
print(f"Combined targeted m/z analysis results saved: {combined_results_path}")

Combined targeted m/z analysis results saved: D:/UCSF_postdoc_topic/ESI_/new_data_acquisition/combined_targeted_mz_results_combined.csv


In [84]:
# open folder and import all .csv file in the designated folder and combined all dataframe
#check and count the duplicated value in compound_id column
import os
import pandas as pd
import numpy as np

# Step 1: Load All CSV Files from the Folder
folder_path = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_499_500_501/peak_area_blank_filtered/targeted_res/'
all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# Step 2: Import and Combine All DataFrames
combined_df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
print(f"Combined DataFrame shape: {combined_df.shape}")

# Step 3: Check and Count Duplicated Values in 'compound_id' Column
duplicated_count = combined_df['compound_id'].duplicated().sum()
print(f"Number of duplicated 'compound_id' values: {duplicated_count}")

# Step 4: Print Duplicated Rows
duplicated_rows = combined_df[combined_df.duplicated(subset='compound_id', keep=False)]
print("Duplicated rows:")
print(duplicated_rows)

combined_df.to_csv('D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_499_500_501/peak_area_blank_filtered/targeted_res/combined_logRF.csv',index=False)

Combined DataFrame shape: (166, 112)
Number of duplicated 'compound_id' values: 1
Duplicated rows:
    Average_MZ     RT  E499_1000x_r1  E499_1000x_r2  E499_1000x_r3  \
5    253.14082  7.006        17551.0        17067.0        20831.0   
39   253.14067  7.005            NaN            NaN            NaN   

    E499_100x_r1  E499_100x_r2  E499_100x_r3  E499_200x_r1  E499_200x_r2  ...  \
5        16054.0       15332.0       14708.0       18666.0       16463.0  ...   
39           NaN           NaN           NaN           NaN           NaN  ...   

    E506_100x_r3  E506_200x_r1  E506_200x_r2  E506_200x_r3  E506_25000x_r1  \
5            NaN           NaN           NaN           NaN             NaN   
39           NaN           NaN           NaN           NaN             NaN   

    E506_25000x_r2  E506_25000x_r3  E506_5000x_r1_r1  E506_5000x_r1_r2  \
5              NaN             NaN               NaN               NaN   
39             NaN             NaN               NaN           

In [54]:
import os
import pandas as pd
import numpy as np

# Step 1: Load Processed Data and Targeted m/z List
# Load the processed LC-MS data file
processed_data_path = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_499_500_501/peak_area_blank_filtered/filtered_feature_table_E499.csv'
processed_data = pd.read_csv(processed_data_path)

# Load the targeted m/z list
targeted_mz_list_path = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_499_500_501/peak_area_blank_filtered/499_batch_target_mzlist.csv'
targeted_mz_list = pd.read_csv(targeted_mz_list_path)

# Step 2: Initialize an Empty DataFrame to Store Results
combined_results = pd.DataFrame()

# Step 3: Iterate Over Each Targeted m/z
for idx, row in targeted_mz_list.iterrows():
    target_mz = row['m/z']
    concentrations = [row['conc1'], row['conc2'], row['conc3'], row['conc4'], row['conc5']]

    # Step 4: Find m/z Matches Within 10 ppm
    ppm_tolerance = 50
    matching_rows = processed_data.copy()
    matching_rows['ppm'] = abs((matching_rows['Average_MZ'] - target_mz) / target_mz) * 1e6
    matching_rows = matching_rows[matching_rows['ppm'] <= ppm_tolerance]

    # Step 5: Add Metadata from Targeted m/z List to Matching Rows
    for col in ['group_id', 'DTXSID', 'compound_id', 'SMILES', 'MOLECULAR_FORMULA', 'm/z']:
        matching_rows[col] = row[col]

    # Step 6: Calculate LogRF for Each Treatment/Concentration Pair with CV Filtering
    logRF_columns = []
    for i, (treatment, conc) in enumerate(zip(["100x", "200x", "1000x", "5000x", "25000x"], concentrations), start=1):
        avg_intensity_col = f"avg_intensity_{treatment}"
        cv_col = f"cv_percent_{treatment}"
        logRF_col = f'LogRF_{i}'
        if avg_intensity_col in matching_rows.columns and cv_col in matching_rows.columns and conc != 0:
            matching_rows[logRF_col] = np.where(matching_rows[cv_col] <= 30, np.log10(matching_rows[avg_intensity_col] / conc), np.nan)
        else:
            matching_rows[logRF_col] = np.nan
        logRF_columns.append(logRF_col)

    # Step 7: Calculate Average LogRF if More Than 3 Values are Available
    matching_rows['average_LogRF'] = matching_rows[logRF_columns].apply(lambda x: x[x.notna()].mean() if x.count() >= 3 else np.nan, axis=1)

    # Step 8: Combine Results
    combined_results = pd.concat([combined_results, matching_rows], ignore_index=True)

# Step 9: Filter Combined Results to Keep Only Rows with Valid average_LogRF and Lowest ppm
filtered_results = combined_results[combined_results['average_LogRF'].notna()]
filtered_results = filtered_results.sort_values(by=['m/z', 'ppm']).drop_duplicates(subset='m/z', keep='first')

# Step 10: Export Combined Results
output_folder = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_499_500_501/peak_area_blank_filtered/targeted_res/'
os.makedirs(output_folder, exist_ok=True)
combined_results_path = os.path.join(output_folder, "combined_targeted_mz_results.csv")
filtered_results.to_csv(combined_results_path, index=False)
print(f"Combined targeted m/z analysis results saved: {combined_results_path}")

Combined targeted m/z analysis results saved: D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_499_500_501/peak_area_blank_filtered/targeted_res/combined_targeted_mz_results.csv


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress, pearsonr

# Step 1: Load the Cleaned Data
# Load the cleaned LC-MS data file
path1 = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_499_500_501/peak_area_blank_filtered/499_500_501_area_filtered.csv'
filtered_data = pd.read_csv(path1)

# Step 2: Separate Data by Groups
# Extract the data for each group separately
unique_groups = filtered_data.columns.str.extract(r'(E\d{3})')[0].dropna().unique()
print(f"Unique groups found: {unique_groups}")

# Step 3: Iterate over each group
for group in unique_groups:
    # Extract group-specific data, keep 'Average_MZ' and 'RT' columns
    group_data = filtered_data.filter(like=group)
    group_data = pd.concat([filtered_data[['Average_MZ', 'RT']], group_data], axis=1)
    print(f"Processing group: {group}, Columns extracted: {group_data.columns.tolist()}")

    # Step 4: Data Cleaning for the Group
    # Remove any columns or rows with a large number of missing values, or impute missing values
    group_data_clean = group_data.dropna(axis=0, thresh=len(group_data.columns) * 0.5)
    group_data_clean = group_data_clean.dropna(axis=1, thresh=len(group_data_clean) * 0.5)
    group_data_clean.fillna(group_data_clean.mean(), inplace=True)
    print(f"Group data cleaned: {group_data_clean.shape}")

    # Step 5: Calculate CV% for Each Treatment and Add to Data
    for treatment in ["100x", "200x", "1000x", "5000x", "25000x"]:
        treatment_cols = [col for col in group_data_clean.columns if f"_{treatment}_" in col]
        if len(treatment_cols) == 0:
            group_data_clean[f"{treatment}_cv"] = np.nan
            continue
        treatment_data = group_data_clean[treatment_cols]
        cv_percent = treatment_data.std(axis=1) / treatment_data.mean(axis=1) * 100
        group_data_clean[f"{treatment}_cv"] = cv_percent
    print(f"Group data with CV% added: {group_data_clean.head()}")

    # Step 6: Filter Data Based on CV% Threshold
    cv_columns = [col for col in group_data_clean.columns if col.endswith('_cv')]
    filtered_group_data = group_data_clean[(group_data_clean[cv_columns] <= 30).sum(axis=1) >= 3]
    print(f"Filtered group data (CV% <= 30 for at least 3 treatments): {filtered_group_data.shape}")

    # Step 7: Export Filtered Data for the Group
    output_folder = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_499_500_501/peak_area_blank_filtered/plot_res/'
    os.makedirs(output_folder, exist_ok=True)
    filtered_data_path = os.path.join(output_folder, f"filtered_feature_table_{group}.csv")
    filtered_group_data.to_csv(filtered_data_path, index=False)
    print(f"Filtered data saved for group {group}: {filtered_data_path}")

# Step 8: Targeted m/z Search and Analysis
# Load the targeted m/z list
mz_list_path = 'D:/UCSF_postdoc_topic/ESI_/ENTACT/first_ENTACT_mixtures_mzlist.csv'
targeted_mz_list = pd.read_csv(mz_list_path)
print(f"Targeted m/z list loaded: {len(targeted_mz_list)} entries")

# Initialize an empty list to store results for the group
results = []

# Iterate over each targeted m/z
for idx, row in targeted_mz_list.iterrows():
    target_mz = row['m/z']
    compound_id = row['compound_id']
    group_id = row['group_id']
    concentrations = [row['conc1'], row['conc2'], row['conc3'], row['conc4'], row['conc5']]

    # Only process if the group_id matches the current group
    if group_id not in unique_groups:
        continue

    group_data_clean = pd.read_csv(os.path.join(output_folder, f"filtered_feature_table_{group_id}.csv"))

    # Find m/z matches within 10 ppm
    ppm_tolerance = 20
    matches = group_data_clean['Average_MZ'].apply(lambda mz: abs((mz - target_mz) / target_mz) * 1e6 <= ppm_tolerance)
    matching_mzs = group_data_clean[matches]
    print(f"Target m/z: {target_mz}, Matches found: {len(matching_mzs)}")

    # Rename m/z if multiple matches are found
    if len(matching_mzs) > 1:
        matching_mzs['Average_MZ'] = [f"{mz}_{i+1}" for i, mz in enumerate(matching_mzs['Average_MZ'])]

    # Calculate average intensity and CV% for each treatment
    avg_intensities = []
    valid_concentrations = []
    cv_percents = []
    for treatment, conc in zip(["100x", "200x", "1000x", "5000x", "25000x"], concentrations):
        treatment_cols = [col for col in matching_mzs.columns if f"_{treatment}_" in col]
        if len(treatment_cols) == 0:
            avg_intensities.append(np.nan)
            valid_concentrations.append(conc)
            cv_percents.append(np.nan)
            continue
        treatment_data = matching_mzs[treatment_cols]
        avg_intensity = treatment_data.mean(axis=1).mean()  # Averaging the intensity across the treatment columns
        cv_percent = (treatment_data.std(axis=1) / treatment_data.mean(axis=1) * 100).mean()  # Average CV%
        avg_intensities.append(avg_intensity)
        valid_concentrations.append(conc)
        cv_percents.append(cv_percent)
    print(f"Average intensities: {avg_intensities}, CV%: {cv_percents}, Valid concentrations: {valid_concentrations}")

    # Remove NaN values for correlation analysis
    filtered_avg_intensities = [val for val in avg_intensities if not np.isnan(val)]
    filtered_concentrations = [conc for val, conc in zip(avg_intensities, valid_concentrations) if not np.isnan(val)]
    filtered_cv_percents = [cv for cv in cv_percents if not np.isnan(cv)]

    # Check if there are enough valid treatments to proceed
    if len(filtered_avg_intensities) < 3:
        print(f"Not enough valid treatments for target m/z {target_mz}")
        continue

    # Count treatments with CV% < 30
    filtered_concentrations = [conc for conc, cv in zip(filtered_concentrations, filtered_cv_percents) if cv < 30]
    filtered_avg_intensities = [val for val, cv in zip(filtered_avg_intensities, filtered_cv_percents) if cv < 30]
    if len(filtered_avg_intensities) < 3:
        print(f"Not enough treatments with CV% < 30 for target m/z {target_mz}")
        continue

    # Perform correlation analysis
    slope, intercept, r_value, p_value, std_err = linregress(filtered_concentrations, filtered_avg_intensities)
    print(f"Correlation analysis for target m/z {target_mz}: Slope: {slope}, R^2: {r_value**2}")

    # Plot scatter plot and regression line
    plt.figure(figsize=(8, 6))
    plt.scatter(filtered_concentrations, filtered_avg_intensities, label='Data Points')
    plt.plot(filtered_concentrations, intercept + slope * np.array(filtered_concentrations), 'r', label=f'Fit Line (R^2={r_value**2:.2f})')
    plt.xlabel('Concentration')
    plt.ylabel('Average Intensity')
    plt.title(f'Scatter Plot and Linear Regression for m/z {target_mz} in Group {group_id}')
    plt.legend()
    plot_path = os.path.join(output_folder, f"scatter_plot_{group_id}_{target_mz}.png")
    plt.savefig(plot_path)
    plt.close()
    print(f"Plot saved for target m/z {target_mz}: {plot_path}")

    # Save results
    results.append({
        "m/z": target_mz,
        "compound_id": compound_id,
        "group_id": group_id,
        "avg_intensities": avg_intensities,
        "cv_percents": cv_percents,
        "cv_below_30_count": len(filtered_avg_intensities),
        "pearson_r2": r_value**2,
        "slope": slope,
        "intercept": intercept
    })

# Step 9: Save Results to DataFrame for All Groups
results_df = pd.DataFrame(results)
results_path = os.path.join(output_folder, "targeted_mz_results_all_groups.csv")
if not results_df.empty:
    results_df.to_csv(results_path, index=False)
    print(f"Results saved for all groups: {results_path}")
else:
    print(f"No results to save for any group.")

In [ ]:
import os
import pandas as pd

# Function to match m/z values from targeted list with Average_MZ column and filter based on ppm tolerance
# do MS1 search, find the # of detected compounds
def match_mz(targeted_mz_list, data, ppm_threshold=20):
    results = []

    for targeted_mz in targeted_mz_list:
        # Calculate ppm difference for each value in Average_MZ column
        data['ppm_diff'] = abs(data['Average_MZ'] - targeted_mz) / targeted_mz * 1e6

        # Filter rows where ppm difference is within the threshold
        filtered_hits = data[data['ppm_diff'] <= ppm_threshold]

        # Append the number of hits for the targeted m/z value
        results.append({'Targeted_MZ': targeted_mz, 'Number_of_Hits': len(filtered_hits)})

    # Create a DataFrame to store the results
    results_df = pd.DataFrame(results)
    return results_df

# Batch processing for multiple files
batches = ['499', '500', '501', '502', '504', '506']
output_results = {}
summary_results = []


for batch in batches:
    # Determine the correct folder for the targeted m/z list based on the batch number
    if batch in ['499', '500', '501']:
        targeted_mz_list_folder = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_499_500_501/peak_area_blank_filtered/'
    else:
        targeted_mz_list_folder = 'D:/UCSF_postdoc_topic/ESI_/DTSC_DATA_ENTACT/ENTACT_502_504_506/peak_area_with_blank_filter/'
    
    # Load processed data and targeted m/z list for each batch
    processed_data_path = os.path.join(input_folder, f'filtered_feature_table_pos_EPA{batch}.csv')
    targeted_mz_list_path = os.path.join(targeted_mz_list_folder, f'{batch}_batch_target_mzlist.csv')
    
    processed_data = pd.read_csv(processed_data_path)
    targeted_mz_list = pd.read_csv(targeted_mz_list_path)

    # Match m/z values and get the number of hits
    results_df = match_mz(targeted_mz_list['m/z'], processed_data, 30)
    num_hits = results_df[results_df['Number_of_Hits'] > 0].shape[0]
    
    output_results[batch] = results_df
    summary_results.append({'Batch': batch, 'Total_Targeted_m/z': len(targeted_mz_list), 'Number_of_Hits': num_hits})

# Create a summary table of the results
summary_df = pd.DataFrame(summary_results)
print(summary_df)

# Example output access
# To access results for a specific batch, use: output_results['499']


  Batch  Total_Targeted_m/z  Number_of_Hits
0   499                  95              75
1   500                  94              70
2   501                  95              71
3   502                  95              75
4   504                 185             148
5   506                 365             275
